In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
df = pd.read_csv('/kaggle/input/week2dl/customer_attrition.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
x = df.iloc[:,3:-1]
y = df.iloc[:,-1]

In [4]:
def enc(df):
    encoder = LabelEncoder()
    df = encoder.fit_transform(df)
    return df

x['Geography'] = enc(x['Geography'])
x['Gender'] = enc(x['Geography'])

In [5]:
col_list = ['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']

def scale(df):
    scaler = StandardScaler()
    df = scaler.fit_transform(df)
    return df

for c in col_list:
    x[c]=scale(x[[c]])

In [6]:
x_train,x_temp,y_train,y_temp = train_test_split(x,y,test_size=0.3, random_state=42)
x_test,x_val,y_test,y_val = train_test_split(x_temp,y_temp,test_size=0.5, random_state=42)

In [7]:
x_train.shape

(7000, 10)

In [8]:
model_list = [
    
    Sequential([
    Input(shape=(x_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
    ]),
    
    Sequential([
    Input(shape=(x_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
    ]),
    
    Sequential([
    Input(shape=(x_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
    ]),
]

epoch_list = [10,20,30]
learning_rate_list = [0.1,0.01,0.001]

In [9]:
results = []

for i in model_list:
    print(i.summary())
    for j in epoch_list:
        for k in learning_rate_list:
            i.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=k), loss='binary_crossentropy', metrics=['accuracy','precision','recall'])
            history = i.fit(x_train,y_train,epochs=j,batch_size=32,validation_data=(x_val,y_val))
            loss, accuracy, precision, recall = i.evaluate(x_test,y_test)
            results.append({'Model':i,'Epochs':j,'Learning Rate':k,'Loss':loss,'Accuracy':accuracy,'Precision':precision,'Recall':recall}) 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,537 (6.00 KB)

 Trainable params: 1,537 (6.00 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,777 (46.00 KB)

 Trainable params: 11,777 (46.00 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7756 - loss: 0.6441 - precision: 0.3259 - recall: 0.0473 - val_accuracy: 0.8000 - val_loss: 0.5006 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8007 - loss: 0.5009 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8000 - val_loss: 0.5023 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7901 - loss: 0.5149 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8000 - val_loss: 0.5008 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7938 - loss: 0.5100 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.8000 - val_loss: 0.5004 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7934 - loss: 0.5114 - precision: 

In [10]:
res = pd.DataFrame(results)
res

,Model,Epochs,Learning Rate,Loss,Accuracy,Precision,Recall
0,"<Sequential name=sequential, built=True>",10,0.100,0.366731,0.864667,0.742515,0.436620
1,"<Sequential name=sequential, built=True>",10,0.010,0.343326,0.869333,0.775000,0.436620
2,"<Sequential name=sequential, built=True>",10,0.001,0.344565,0.868667,0.780645,0.426056
3,"<Sequential name=sequential, built=True>",20,0.100,0.361892,0.857333,0.850000,0.299296
4,"<Sequential name=sequential, built=True>",20,0.010,0.343290,0.867333,0.793103,0.404930
5,"<Sequential name=sequential, built=True>",20,0.001,0.343093,0.865333,0.773333,0.408451
6,"<Sequential name=sequential, built=True>",30,0.100,0.377072,0.858000,0.738255,0.387324
7,"<Sequential name=sequential, built=True>",30,0.010,0.344100,0.862667,0.774648,0.387324
8,"<Sequential name=sequential, built=True>",30,0.001,0.344908,0.866000,0.778524,0.408451
9,"<Sequential name=sequential_1, built=True>",10,0.100,0.486018,0.810667,0.000000,0.000000
